In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd
import dask.dataframe as dd

## Functions

In [2]:
def file_to_dataframe(file_name, sheet_num=0):
    """Open file as dataframe and check execution

    Args:
        file_name (string): file name
        sheet_num (integer): index of individual sheet to retrieve
    """
    try:
        df = pd.read_excel(file_name, sheet_name=sheet_num)
    except:
        raise Exception(f"EXCEPTION: File Not Found -> Dataframe not created for {file_name}")
    else:
        return df
        

## Processing

In [39]:
RSL_Data_df = file_to_dataframe('Lambeck_Farfield_Adjusted.xlsx', 0)
References_df = file_to_dataframe('Lambeck_Farfield_Adjusted.xlsx', 1)
Additional_Info_df = file_to_dataframe('Lambeck_Farfield_Adjusted.xlsx', 2)

## Cleaning up RSL_Data_df, had some duplicate rows, don't need observation numbers
RSL_Data_df.drop(['Observation Number'], axis = 1, inplace=True)
RSL_Data_df.drop_duplicates(inplace=True)
RSL_Data_df.reset_index(inplace=True)
RSL_Data_df.drop(['index'], axis = 1, inplace=True)

RSL_copy_ref = RSL_Data_df['Reference_Number'].copy()
for i in range(len(RSL_copy_ref)):
    if isinstance(RSL_copy_ref[i], str):
        RSL_copy_ref[i] = np.array([int(x) for x in RSL_copy_ref[i].split('/')])
RSL_Data_df['Reference_Number']=RSL_copy_ref

#RSL_Data_df = RSL_Data_df.set_index(['Site_Location','Age_ka'])
RSL_Data_df

,Site_Location,Latitude,Longitude,Age_ka,RSL_m,Sigma_RSL_m,Sample_Type,Reference_Number
0,Tahiti,-17.45,210.45,7.210,-0.5,5.1,Coral,"[1, 2]"
1,Tahiti,-17.45,210.45,7.670,-4.1,5.1,Coral,"[1, 2]"
2,Tahiti,-17.45,210.45,7.780,-5.6,5.1,Coral,"[1, 2]"
3,Tahiti,-17.45,210.45,8.580,-18.8,5.1,Coral,"[1, 2]"
4,Tahiti,-17.45,210.45,8.790,-14.5,5.1,Coral,"[1, 2]"
...,...,...,...,...,...,...,...,...
960,Barbados,13.04,300.45,19.708,-108.0,3.3,A. palmate,"[72, 73]"
961,Barbados,13.04,300.45,30.147,-85.6,4.1,A. palmate,"[72, 73]"
962,Barbados,13.04,300.45,30.225,-86.0,4.1,A. palmate,"[72, 73]"
963,Barbados,13.04,300.45,30.242,-86.6,4.1,A. palmate,"[72, 73]"


In [42]:

RSL_Adj = RSL_Data_df.groupby(['Site_Location', 'Latitude', 'Longitude', 'Age_ka', 'Sample_Type'])['RSL_m'].apply(np.array).reset_index(name = 'RSL_m')
Sigma_RSL_Adj = RSL_Data_df.groupby(['Site_Location', 'Latitude', 'Longitude', 'Age_ka', 'Sample_Type'])['Sigma_RSL_m'].apply(np.array).reset_index(name = 'Sigma_RSL_m')
References_Adj = RSL_Data_df.groupby(['Site_Location', 'Latitude', 'Longitude', 'Age_ka', 'Sample_Type'])['Reference_Number'].apply(np.array).reset_index(name = 'Reference_Number')

RSL_Data_Adj = (RSL_Adj.merge(Sigma_RSL_Adj).merge(References_Adj))[RSL_Data_df.columns.to_list()]

'''
## Cleanup Reference Number
def adj_ref_list(ref_nums):
    ref_nums_adj = ref_nums.copy()
    for i, elem in enumerate(ref_nums):
        elem_temp = elem.copy()
        for j, entry in enumerate(elem):
            if isinstance(entry, str):
                elem_temp[j] = np.array([int(num) for num in entry.split('/')])
        ref_nums_adj[i] = np.array(elem_temp)
    return ref_nums_adj

# Apply the adj_ref_list function, does not edit in place (though that was a pain to figure out haha!)
RSL_Data_Adj['Reference Number'] = adj_ref_list(RSL_Data_Adj['Reference Number'])
RSL_Data_Adj['Reference Number']

'''

RSL_Data_Adj

,Site_Location,Latitude,Longitude,Age_ka,RSL_m,Sigma_RSL_m,Sample_Type,Reference_Number
0,Abrolhos,-28.68,113.83,1.058,[0.5],[2.1],Coral,"[[11, 12]]"
1,Abrolhos,-28.68,113.83,2.335,[0.7],[2.1],Coral,"[[11, 12]]"
2,Abrolhos,-28.68,113.83,3.226,[0.7],[2.1],Coral,"[[11, 12]]"
3,Abrolhos,-28.68,113.83,3.831,[0.8],[2.1],Coral,"[[11, 12]]"
4,Abrolhos,-28.68,113.83,3.867,[1.0],[2.1],Coral,"[[11, 12]]"
...,...,...,...,...,...,...,...,...
952,Zone_B_Dadalla,6.04,80.18,5.785,[0.5],[0.5],Sediment,[64]
953,Zone_C_Mihiripenna,6.01,80.26,5.738,[0.8],[0.5],Sediment,[64]
954,Zone_D_Aranwala,5.97,80.38,5.855,[0.7],[0.5],Sediment,[64]
955,Zone_E_Pallikkudawa,6.02,80.79,5.773,[1.1],[0.5],Sediment,[64]


In [43]:
#RSL_Data = RSL_Data_Adj.set_index(['Site Location']).to_xarray()
RSL_Data = RSL_Data_Adj.to_xarray()
RSL_Data = RSL_Data.set_coords(['Site_Location', 'Latitude', 'Longitude', 'Age_ka'])
RSL_Data = RSL_Data.set_xindex(['Site_Location', 'Latitude', 'Longitude', 'Age_ka'])
RSL_Data = RSL_Data.rename_dims({'index':'MultiIndex'})
RSL_Data

<xarray.Dataset> Size: 69kB
Dimensions:           (MultiIndex: 957)
Coordinates:
  * index             (MultiIndex) object 8kB MultiIndex
  * Site_Location     (MultiIndex) object 8kB 'Abrolhos' ... 'Zone_E_Pallikku...
  * Latitude          (MultiIndex) float64 8kB -28.68 -28.68 ... 6.02 6.02
  * Longitude         (MultiIndex) float64 8kB 113.8 113.8 113.8 ... 80.79 80.79
  * Age_ka            (MultiIndex) float64 8kB 1.058 2.335 3.226 ... 5.773 5.936
Dimensions without coordinates: MultiIndex
Data variables:
    RSL_m             (MultiIndex) object 8kB 0.5 0.7 0.7 0.8 ... 0.7 1.1 0.8
    Sigma_RSL_m       (MultiIndex) object 8kB 2.1 2.1 2.1 2.1 ... 0.5 0.5 0.5
    Sample_Type       (MultiIndex) object 8kB 'Coral' 'Coral' ... 'Sediment'
    Reference_Number  (MultiIndex) object 8kB [array([11, 12])] ... [64]

In [48]:
RSL_Data.drop_vars(['Reference_Number']).Site_Location

<xarray.DataArray 'Site_Location' (MultiIndex: 957)> Size: 8kB
array(['Abrolhos', 'Abrolhos', 'Abrolhos', ..., 'Zone_D_Aranwala',
       'Zone_E_Pallikkudawa', 'Zone_E_Pallikkudawa'], dtype=object)
Coordinates:
  * index          (MultiIndex) object 8kB MultiIndex
  * Site_Location  (MultiIndex) object 8kB 'Abrolhos' ... 'Zone_E_Pallikkudawa'
  * Latitude       (MultiIndex) float64 8kB -28.68 -28.68 -28.68 ... 6.02 6.02
  * Longitude      (MultiIndex) float64 8kB 113.8 113.8 113.8 ... 80.79 80.79
  * Age_ka         (MultiIndex) float64 8kB 1.058 2.335 3.226 ... 5.773 5.936
Dimensions without coordinates: MultiIndex

In [51]:
RSL_Data.sel({'Site_Location':'Kiritimati', 'Age_ka':1.599})['RSL_m']

ValueError: can only convert an array of size 1 to a Python scalar

<xarray.DataArray 'RSL_m' (MultiIndex: 1)> Size: 8B
array([array([0. , 0.1])], dtype=object)
Coordinates:
  * index          (MultiIndex) object 8B MultiIndex
  * Latitude       (MultiIndex) float64 8B 1.98
  * Longitude      (MultiIndex) float64 8B 202.5
  * MultiIndex     (MultiIndex) object 8B MultiIndex
    Site_Location  <U10 40B 'Kiritimati'
    Age_ka         float64 8B 1.599

Duplicates that were removed (kept only one of each)
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Site Location</th>
      <th>Latitude (degree)</th>
      <th>Longitude (degree)</th>
      <th>Age (ka)</th>
      <th>RSL (m)</th>
      <th>Sigma RSL (m)</th>
      <th>Sample Type</th>
      <th>Reference Number</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>34</th>
      <td>Kiritimati</td>
      <td>1.98</td>
      <td>202.52</td>
      <td>0.090</td>
      <td>-0.1</td>
      <td>0.1</td>
      <td>Coral</td>
      <td>3</td>
    </tr>
    <tr>
      <th>35</th>
      <td>Kiritimati</td>
      <td>1.98</td>
      <td>202.52</td>
      <td>0.090</td>
      <td>-0.1</td>
      <td>0.1</td>
      <td>Coral</td>
      <td>3</td>
    </tr>
    <tr>
      <th>602</th>
      <td>Geylang</td>
      <td>1.31</td>
      <td>103.87</td>
      <td>8.795</td>
      <td>-8.8</td>
      <td>0.7</td>
      <td>Sediment</td>
      <td>28 / 29</td>
    </tr>
    <tr>
      <th>603</th>
      <td>Geylang</td>
      <td>1.31</td>
      <td>103.87</td>
      <td>8.795</td>
      <td>-8.8</td>
      <td>0.7</td>
      <td>Sediment</td>
      <td>28 / 29</td>
    </tr>
    <tr>
      <th>820</th>
      <td>Maldives Rasdhoo</td>
      <td>4.30</td>
      <td>72.98</td>
      <td>7.725</td>
      <td>-4.9</td>
      <td>3.0</td>
      <td>Coral</td>
      <td>65</td>
    </tr>
    <tr>
      <th>821</th>
      <td>Maldives Rasdhoo</td>
      <td>4.30</td>
      <td>72.98</td>
      <td>7.725</td>
      <td>-4.9</td>
      <td>3.0</td>
      <td>Coral</td>
      <td>65</td>
    </tr>
  </tbody>
</table>
</div>

Overlapping Ages/Locations that were merged together

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Site Location</th>
      <th>Latitude (degree)</th>
      <th>Longitude (degree)</th>
      <th>Age (ka)</th>
      <th>RSL (m)</th>
      <th>Sigma RSL (m)</th>
      <th>Sample Type</th>
      <th>Reference Number</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>59</th>
      <td>Kiritimati</td>
      <td>1.98</td>
      <td>202.52</td>
      <td>1.599</td>
      <td>0.0</td>
      <td>0.1</td>
      <td>Coral</td>
      <td>3</td>
    </tr>
    <tr>
      <th>60</th>
      <td>Kiritimati</td>
      <td>1.98</td>
      <td>202.52</td>
      <td>1.599</td>
      <td>0.1</td>
      <td>0.1</td>
      <td>Coral</td>
      <td>3</td>
    </tr>
    <tr>
      <th>77</th>
      <td>Kiritimati</td>
      <td>1.98</td>
      <td>202.52</td>
      <td>1.880</td>
      <td>0.0</td>
      <td>0.1</td>
      <td>Coral</td>
      <td>3</td>
    </tr>
    <tr>
      <th>78</th>
      <td>Kiritimati</td>
      <td>1.98</td>
      <td>202.52</td>
      <td>1.880</td>
      <td>0.1</td>
      <td>0.1</td>
      <td>Coral</td>
      <td>3</td>
    </tr>
    <tr>
      <th>95</th>
      <td>Kiritimati</td>
      <td>1.98</td>
      <td>202.52</td>
      <td>2.749</td>
      <td>-0.1</td>
      <td>0.1</td>
      <td>Coral</td>
      <td>3</td>
    </tr>
    <tr>
      <th>96</th>
      <td>Kiritimati</td>
      <td>1.98</td>
      <td>202.52</td>
      <td>2.749</td>
      <td>0.1</td>
      <td>0.1</td>
      <td>Coral</td>
      <td>3</td>
    </tr>
    <tr>
      <th>109</th>
      <td>Kiritimati</td>
      <td>1.98</td>
      <td>202.52</td>
      <td>3.550</td>
      <td>0.2</td>
      <td>0.1</td>
      <td>Coral</td>
      <td>3</td>
    </tr>
    <tr>
      <th>110</th>
      <td>Kiritimati</td>
      <td>1.98</td>
      <td>202.52</td>
      <td>3.550</td>
      <td>0.0</td>
      <td>0.1</td>
      <td>Coral</td>
      <td>3</td>
    </tr>
    <tr>
      <th>514</th>
      <td>Sunda-18300-2</td>
      <td>4.36</td>
      <td>108.65</td>
      <td>14.350</td>
      <td>-91.6</td>
      <td>3.0</td>
      <td>Sediment</td>
      <td>26 / 27</td>
    </tr>
    <tr>
      <th>515</th>
      <td>Sunda-18300-2</td>
      <td>4.36</td>
      <td>108.65</td>
      <td>14.350</td>
      <td>-95.3</td>
      <td>3.0</td>
      <td>Sediment</td>
      <td>26 / 27</td>
    </tr>
    <tr>
      <th>630</th>
      <td>TNTH</td>
      <td>7.75</td>
      <td>100.17</td>
      <td>7.787</td>
      <td>-4.0</td>
      <td>0.9</td>
      <td>Mangrove</td>
      <td>38</td>
    </tr>
    <tr>
      <th>632</th>
      <td>TNTH</td>
      <td>7.75</td>
      <td>100.17</td>
      <td>7.787</td>
      <td>-4.0</td>
      <td>0.6</td>
      <td>Mangrove</td>
      <td>40</td>
    </tr>
    <tr>
      <th>851</th>
      <td>Maldives Maalhosmadulu</td>
      <td>5.27</td>
      <td>73.03</td>
      <td>7.424</td>
      <td>-3.1</td>
      <td>3.0</td>
      <td>Coral</td>
      <td>66</td>
    </tr>
    <tr>
      <th>852</th>
      <td>Maldives Maalhosmadulu</td>
      <td>5.27</td>
      <td>73.03</td>
      <td>7.424</td>
      <td>-4.5</td>
      <td>3.0</td>
      <td>Coral</td>
      <td>66</td>
    </tr>
    <tr>
      <th>897</th>
      <td>Mayotte</td>
      <td>-12.80</td>
      <td>45.27</td>
      <td>8.200</td>
      <td>-5.9</td>
      <td>2.5</td>
      <td>Coral</td>
      <td>69</td>
    </tr>
    <tr>
      <th>898</th>
      <td>Mayotte</td>
      <td>-12.80</td>
      <td>45.27</td>
      <td>8.200</td>
      <td>-8.1</td>
      <td>2.5</td>
      <td>Coral</td>
      <td>69</td>
    </tr>
  </tbody>
</table>
</div>